<a href="https://colab.research.google.com/github/TomMcIver/Stock/blob/Data-CleanUp/stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ace-tools yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import time
import os
from datetime import datetime, timedelta

TECH_STOCKS = [
    "AAPL", "MSFT", "NVDA", "AVGO", "CRM", "ORCL", "CSCO", "ACN", "NOW", "IBM",
    "AMD", "ADBE", "INTU", "QCOM", "TXN", "AMAT", "PLTR", "ANET", "PANW", "MU",
    "ADI", "LRCX", "KLAC", "APH", "INTC", "CDNS", "CRWD", "MSI", "SNPS", "ADSK",
    "FTNT", "ROP", "WDAY", "NXPI", "FICO", "TEL", "IT", "CTSH", "GLW", "DELL",
    "HPQ", "MCHP", "ANSS", "MPWR", "HPE", "KEYS", "GDDY", "CDW", "TYL", "NTAP"
]

MACRO_TICKERS = {
    '^VIX': 'Volatility Index',
    '^TNX': '10-Year Treasury Yield',
    '^FVX': '5-Year Treasury Yield',
    'CL=F': 'Crude Oil Futures'
}

DATA_FOLDER = "yfinance_tech_stock_data"
MACRO_FOLDER = "macroeconomic_data"
os.makedirs(DATA_FOLDER, exist_ok=True)
os.makedirs(MACRO_FOLDER, exist_ok=True)

def fetch_stock_data(symbol, interval="1h", period="2y"):
    """Fetch and save stock data with enhanced error handling"""
    try:
        print(f"\n📈 Fetching {symbol} ({interval} interval)...")
        df = yf.Ticker(symbol).history(period=period, interval=interval)

        if df.empty:
            print(f"🚨 No data for {symbol}")
            return None

        df = df.reset_index().rename(columns={"Datetime": "timestamp"})
        df['symbol'] = symbol


        path = os.path.join(DATA_FOLDER, f"{symbol}_stock.csv")
        df.to_csv(path, index=False)
        print(f"✅ Saved {len(df)} rows to {path}")

        return df

    except Exception as e:
        print(f"🚨 Error fetching {symbol}: {str(e)}")
        return None

def fetch_macro_data():
    """Fetch macroeconomic indicators with retry logic"""
    macro_data = {}

    for ticker, name in MACRO_TICKERS.items():
        retries = 3
        while retries > 0:
            try:
                print(f"\n🌐 Fetching {name} ({ticker})...")
                df = yf.download(ticker, start=datetime.now()-timedelta(days=730), interval='1h')

                if not df.empty:
                    df = df.reset_index().rename(columns={'Date': 'timestamp'})
                    df['ticker'] = ticker
                    macro_data[ticker] = df


                    path = os.path.join(MACRO_FOLDER, f"{ticker.replace('^','')}.csv")
                    df.to_csv(path, index=False)
                    print(f"✅ Saved {ticker} data")


                    print(f"\n📋 Columns for {ticker} ({name}):")
                    print(df.columns.tolist())
                    print(f"\nFirst row for {ticker}:")
                    print(df.head(1))
                    break
                else:
                    print(f"🚨 Empty data for {ticker}")
                    retries -= 1

            except Exception as e:
                print(f"🚨 Error ({retries} retries left): {str(e)}")
                retries -= 1
                time.sleep(5)

    return macro_data

def main_data_pipeline():
    """Orchestrate data fetching with rate limiting"""
    stock_dfs = []
    preview_printed = False


    for symbol in TECH_STOCKS:
        df = fetch_stock_data(symbol)
        if df is not None:
            stock_dfs.append(df)
            if not preview_printed:
                print(f"\n📋 Columns for {symbol}:")
                print(df.columns.tolist())
                print(f"\nFirst row for {symbol}:")
                print(df.head(1))
                preview_printed = True
        time.sleep(1.5)


    macro_data = fetch_macro_data()

    return pd.concat(stock_dfs), macro_data


if __name__ == "__main__":
    stock_data, macro_data = main_data_pipeline()
    print("\n🎉 Data pipeline complete!")
    print(f"Stock data shape: {stock_data.shape}")
    print(f"Macro data keys: {list(macro_data.keys())}")


📈 Fetching AAPL (1h interval)...
✅ Saved 3494 rows to yfinance_tech_stock_data/AAPL_stock.csv

📋 Columns for AAPL:
['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'symbol']

First row for AAPL:
                  timestamp        Open        High         Low       Close  \
0 2023-02-15 09:30:00-05:00  153.110001  154.520004  152.880005  153.720001   

     Volume  Dividends  Stock Splits symbol  
0  17518299        0.0           0.0   AAPL  

📈 Fetching MSFT (1h interval)...
✅ Saved 3494 rows to yfinance_tech_stock_data/MSFT_stock.csv

📈 Fetching NVDA (1h interval)...
✅ Saved 3494 rows to yfinance_tech_stock_data/NVDA_stock.csv

📈 Fetching AVGO (1h interval)...
✅ Saved 3494 rows to yfinance_tech_stock_data/AVGO_stock.csv

📈 Fetching CRM (1h interval)...
✅ Saved 3494 rows to yfinance_tech_stock_data/CRM_stock.csv

📈 Fetching ORCL (1h interval)...
✅ Saved 3494 rows to yfinance_tech_stock_data/ORCL_stock.csv

📈 Fetching CSCO (1h interval)...
✅ Saved 34

[*********************100%***********************]  1 of 1 completed


✅ Saved ^VIX data

📋 Columns for ^VIX (Volatility Index):
[('Datetime', ''), ('Close', '^VIX'), ('High', '^VIX'), ('Low', '^VIX'), ('Open', '^VIX'), ('Volume', '^VIX'), ('ticker', '')]

First row for ^VIX:
Price                   Datetime      Close   High    Low       Open Volume  \
Ticker                                 ^VIX   ^VIX   ^VIX       ^VIX   ^VIX   
0      2023-02-20 08:00:00+00:00  21.030001  21.07  20.99  21.059999      0   

Price  ticker  
Ticker         
0        ^VIX  

🌐 Fetching 10-Year Treasury Yield (^TNX)...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

✅ Saved ^TNX data

📋 Columns for ^TNX (10-Year Treasury Yield):
[('Datetime', ''), ('Close', '^TNX'), ('High', '^TNX'), ('Low', '^TNX'), ('Open', '^TNX'), ('Volume', '^TNX'), ('ticker', '')]

First row for ^TNX:
Price                   Datetime  Close   High    Low   Open Volume ticker
Ticker                             ^TNX   ^TNX   ^TNX   ^TNX   ^TNX       
0      2023-02-21 13:20:00+00:00  3.896  3.906  3.881  3.894      0   ^TNX

🌐 Fetching 5-Year Treasury Yield (^FVX)...


✅ Saved ^FVX data

📋 Columns for ^FVX (5-Year Treasury Yield):
[('Datetime', ''), ('Close', '^FVX'), ('High', '^FVX'), ('Low', '^FVX'), ('Open', '^FVX'), ('Volume', '^FVX'), ('ticker', '')]

First row for ^FVX:
Price                   Datetime  Close   High    Low   Open Volume ticker
Ticker                             ^FVX   ^FVX   ^FVX   ^FVX   ^FVX       
0      2023-02-21 13:20:00+00:00  4.113  4.119  4.094  4.103      0   ^FVX

🌐 Fetching Crude Oil Futures (CL=F)...


[*********************100%***********************]  1 of 1 completed


✅ Saved CL=F data

📋 Columns for CL=F (Crude Oil Futures):
[('Datetime', ''), ('Close', 'CL=F'), ('High', 'CL=F'), ('Low', 'CL=F'), ('Open', 'CL=F'), ('Volume', 'CL=F'), ('ticker', '')]

First row for CL=F:
Price                   Datetime      Close       High        Low       Open  \
Ticker                                 CL=F       CL=F       CL=F       CL=F   
0      2023-02-19 23:00:00+00:00  76.489998  76.610001  76.410004  76.529999   

Price  Volume ticker  
Ticker   CL=F         
0          20   CL=F  

🎉 Data pipeline complete!
Stock data shape: (174699, 9)
Macro data keys: ['^VIX', '^TNX', '^FVX', 'CL=F']


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys
import logging
import re
from datetime import datetime, timedelta
from typing import List, Dict
from concurrent.futures import ThreadPoolExecutor, as_completed

TECH_STOCKS = [
    "AAPL", "MSFT", "NVDA", "AVGO", "CRM", "ORCL", "CSCO", "ACN", "NOW", "IBM",
    "AMD", "ADBE", "INTU", "QCOM", "TXN", "AMAT", "PLTR", "ANET", "PANW", "MU",
    "ADI", "LRCX", "KLAC", "APH", "INTC", "CDNS", "CRWD", "MSI", "SNPS", "ADSK",
    "FTNT", "ROP", "WDAY", "NXPI", "FICO", "TEL", "IT", "CTSH", "GLW", "DELL",
    "HPQ", "MCHP", "ANSS", "MPWR", "HPE", "KEYS", "GDDY", "CDW", "TYL", "NTAP"
]

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class ProgressTracker:
    def __init__(self, total_months: int):
        self.start_time = time.time()
        self.total_months = total_months
        self.completed_months = 0
        self.total_articles = 0
        self.current_month = None

    def update_progress(self):
        elapsed = time.time() - self.start_time
        avg_time = elapsed / (self.completed_months + 1e-6)
        remaining = avg_time * (self.total_months - self.completed_months)

        progress = (
            f"\n📅 Current: {self.current_month} | "
            f"Completed: {self.completed_months}/{self.total_months} months | "
            f"Articles: {self.total_articles} | "
            f"Elapsed: {timedelta(seconds=int(elapsed))} | "
            f"ETA: {timedelta(seconds=int(remaining))}"
        )
        sys.stdout.write("\r\033[K" + progress)
        sys.stdout.flush()

class VoxScraper:
    def __init__(self, start_date: datetime, end_date: datetime):
        self.base_url = "https://www.vox.com"
        self.rate_limiter = RateLimiter(calls=3, period=1)
        self.article_buffer = []
        self.all_articles = []  # Stores all articles across months
        self.start_date = start_date
        self.end_date = end_date
        self.total_months = ((end_date.year - start_date.year) * 12
                            + end_date.month - start_date.month + 1)
        self.progress = ProgressTracker(self.total_months)
        self.financial_terms = re.compile(
            r'\b(stock|market|tech|economy|investment|trading|'
            r'earnings|valuation|merger|acquisition|IPO)\b',
            re.IGNORECASE
        )

    def fetch_article_content(self, url: str) -> Dict:
        with self.rate_limiter:
            try:
                response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, 'html.parser')

                time_tag = soup.find('time')
                article_date = pd.to_datetime(time_tag['datetime']) if time_tag else None

                article_body = soup.find('div', class_='c-entry-content')
                text_content = ' '.join([p.get_text() for p in article_body.find_all('p')]) if article_body else ''

                self.progress.total_articles += 1
                self.progress.update_progress()

                return {
                    'url': url,
                    'timestamp': article_date,
                    'content': text_content,
                    'is_financial': bool(self.financial_terms.search(text_content)),
                    'tickers': self._extract_tickers(text_content)
                }

            except Exception as e:
                logging.error(f"Failed to fetch {url}: {str(e)}")
                return None

    def _extract_tickers(self, text: str) -> List[str]:
        return [ticker for ticker in TECH_STOCKS if re.search(r'\b' + ticker + r'\b', text)]

    def scrape_time_range(self):
        current_date = self.start_date
        while current_date <= self.end_date:
            year = current_date.year
            month = current_date.month
            self.progress.current_month = f"{year}-{month:02d}"
            self.progress.update_progress()

            self._scrape_month(year, month)
            current_date += timedelta(days=32)
            self.progress.completed_months += 1
            self.progress.update_progress()

    def _scrape_month(self, year: int, month: int):
        page = 1
        while True:
            url = f"{self.base_url}/archives/{year}/{month}/{page}"
            try:
                article_links = self._get_article_links(url)
                if not article_links:
                    break

                with ThreadPoolExecutor(max_workers=4) as executor:
                    futures = [executor.submit(self.fetch_article_content, link) for link in article_links]
                    for future in as_completed(futures):
                        if (result := future.result()) is not None:
                            self.article_buffer.append(result)

                page += 1
                time.sleep(0.5)

            except Exception as e:
                logging.error(f"Stopping scrape for {year}-{month:02d}: {str(e)}")
                break

        self._save_buffer(year, month)

    def _get_article_links(self, url: str) -> List[str]:
        try:
            response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            return [
                self.base_url + a['href']
                for a in soup.select('div._1p9ghgz3 a[href^="/"]')
                if not a['href'].startswith("/archives")
            ]
        except Exception as e:
            logging.error(f"Failed to get links from {url}: {str(e)}")
            return []

    def _save_buffer(self, year: int, month: int):
        if not self.article_buffer:
            return

        df = pd.DataFrame(self.article_buffer)
        self.all_articles.extend(self.article_buffer)

        df['date_hour'] = df['timestamp'].dt.floor('h')
        df = df.dropna(subset=['date_hour'])

        path = f"vox_articles_{year}_{month:02d}.parquet"
        df.to_parquet(path, index=False)
        logging.info(f"\n✅ Saved {len(df)} articles to {path}")
        self.article_buffer.clear()

class RateLimiter:
    def __init__(self, calls: int, period: int):
        self.calls = calls
        self.period = period
        self.timestamps = []

    def __enter__(self):
        now = time.time()
        self.timestamps = [t for t in self.timestamps if t > now - self.period]

        if len(self.timestamps) >= self.calls:
            sleep_time = self.period - (now - self.timestamps[0])
            time.sleep(sleep_time)

        self.timestamps.append(time.time())

    def __exit__(self, *args):
        pass

if __name__ == "__main__":
    start_date = datetime(2023, 1, 1)
    end_date = datetime.now()

    scraper = VoxScraper(start_date, end_date)
    print("🚀 Starting Vox.com scraper with progress tracking:")
    print(f"⏳ Time range: {start_date.strftime('%Y-%m')} to {end_date.strftime('%Y-%m')}")
    print("📊 Progress will be updated in real-time below:\n")

    try:
        scraper.scrape_time_range()

        final_df = pd.DataFrame(scraper.all_articles)

        print("\n\n🎉 Final Report:")
        print(f"Total articles collected: {len(final_df)}")

        if not final_df.empty:
            stock_articles = final_df[final_df['tickers'].apply(len) > 0]
            print(f"\n📈 Tech Stock Articles Found ({len(stock_articles)}):")
            for idx, row in stock_articles.iterrows():
                print(f"\n📰 Article {idx + 1}:")
                print(f"   URL: {row['url']}")
                print(f"   Date: {row['timestamp'].strftime('%Y-%m-%d') if pd.notnull(row['timestamp']) else 'Unknown'}")
                print(f"   Tickers: {', '.join(row['tickers'])}")
                print(f"   Financial Terms Found: {'Yes' if row['is_financial'] else 'No'}")
            print(f"\n💾 Full data saved to vox_articles_YYYY_MM.parquet files")
        else:
            print("\nNo articles found in the specified date range")

        print(f"\nProcessed months: {scraper.progress.completed_months}/{scraper.total_months}")

    except KeyboardInterrupt:
        print("\n\n⚠️ Scraping interrupted by user! Partial results saved.")

🚀 Starting Vox.com scraper with progress tracking:
⏳ Time range: 2023-01 to 2025-02
📊 Progress will be updated in real-time below:



































📅 Current: 2023-01 | Completed: 0/26 months | Articles: 33 | Elapsed: 0:00:18 | ETA: 5532 days, 12:46:45

ERROR:root:Failed to fetch https://www.vox.com/policy-and-politics/2023/1/25/23571655/monterey-park-half-moon-bay-shooter-old-age-crime: ('Connection broken: IncompleteRead(28938 bytes read, 10523 more expected)', IncompleteRead(28938 bytes read, 10523 more expected))
















































































































































































📅 Current: 2023-02 | Completed: 1/26 months | Articles: 205 | Elapsed: 0:01:40 | ETA: 0:41:51

ERROR:root:Failed to fetch https://www.vox.com/culture/23612887/ant-man-quantumania-marvel-flop-era: HTTPSConnectionPool(host='www.vox.com', port=443): Read timed out. (read timeout=10)































📅 Current: 2023-02 | Completed: 1/26 months | Articles: 234 | Elapsed: 0:01:59 | ETA: 0:49:45

...











































































📅 Current: 2024-11 | Completed: 21/26 months | Articles: 4017 | Elapsed: 0:31:01 | ETA: 0:07:23

ERROR:root:Failed to fetch https://www.vox.com/technology/385433/what-is-bluesky-twitter-social-media-use-future: HTTPSConnectionPool(host='www.vox.com', port=443): Read timed out. (read timeout=10)






























































📅 Current: 2024-11 | Completed: 21/26 months | Articles: 4077 | Elapsed: 0:31:27 | ETA: 0:07:29

ERROR:root:Failed to get links from https://www.vox.com/archives/2024/11/13: HTTPSConnectionPool(host='www.vox.com', port=443): Read timed out. (read timeout=10)












































































































































































































































































































































































































































































📅 Current: 2025-02 | Completed: 25/26 months | Articles: 4528 | Elapsed: 0:34:03 | ETA: 0:01:21

🎉 Final Report:
Total articles collected: 4528

📈 Tech Stock Articles Found (0):

💾 Full data saved to vox_articles_YYYY_MM.parquet files

Processed months: 25/26


In [ ]:
import os
import pandas as pd
from datetime import timezone

def merge_and_clean_parquet():
    """Merge all Parquet files and delete individual files"""


    parquet_files = [f for f in os.listdir()
                    if f.startswith("vox_articles_")
                    and f.endswith(".parquet")]

    if not parquet_files:
        print("❌ No Parquet files found to merge")
        return

    print(f"🔍 Found {len(parquet_files)} files to merge")


    merged_df = pd.DataFrame()
    for idx, file in enumerate(parquet_files, 1):
        try:
            df = pd.read_parquet(file)
            merged_df = pd.concat([merged_df, df], ignore_index=True)
            print(f"✅ Merged {file} ({len(df)} articles)")
        except Exception as e:
            print(f"🚨 Error merging {file}: {str(e)}")

    if merged_df.empty:
        print("❌ No data to save")
        return


    merged_file = "merged_vox_articles.parquet"
    merged_df.to_parquet(merged_file, index=False)
    print(f"\n💾 Saved merged data to {merged_file}")
    print(f"📊 Total articles: {len(merged_df):,}")

    if input("\n🚨 Delete original files? (y/n): ").lower() == 'y':
        for file in parquet_files:
            try:
                os.remove(file)
                print(f"🗑️ Deleted {file}")
            except Exception as e:
                print(f"🚨 Error deleting {file}: {str(e)}")
        print("\n✅ Cleanup complete")
    else:
        print("\n🔵 Original files preserved")

if __name__ == "__main__":
    merge_and_clean_parquet()


🔍 Found 25 files to merge
✅ Merged vox_articles_2023_10.parquet (182 articles)
✅ Merged vox_articles_2024_05.parquet (177 articles)
✅ Merged vox_articles_2024_06.parquet (200 articles)
✅ Merged vox_articles_2023_02.parquet (179 articles)
✅ Merged vox_articles_2025_02.parquet (115 articles)
✅ Merged vox_articles_2023_07.parquet (186 articles)
✅ Merged vox_articles_2025_01.parquet (181 articles)
✅ Merged vox_articles_2023_09.parquet (200 articles)
✅ Merged vox_articles_2023_11.parquet (150 articles)
✅ Merged vox_articles_2024_04.parquet (168 articles)
✅ Merged vox_articles_2023_04.parquet (227 articles)
✅ Merged vox_articles_2023_06.parquet (216 articles)
✅ Merged vox_articles_2024_08.parquet (104 articles)
✅ Merged vox_articles_2023_12.parquet (137 articles)
✅ Merged vox_articles_2024_01.parquet (150 articles)
✅ Merged vox_articles_2024_12.parquet (154 articles)
✅ Merged vox_articles_2024_10.parquet (209 articles)
✅ Merged vox_articles_2023_03.parquet (212 articles)
✅ Merged vox_article

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import sys
from concurrent.futures import ThreadPoolExecutor, as_completed


INPUT_FILE = "/content/merged_vox_articles.parquet"
OUTPUT_FILE = "/content/scraped_vox_articles.parquet"

def format_eta(seconds):
    """Convert seconds to HH:MM:SS format"""
    return time.strftime("%H:%M:%S", time.gmtime(seconds))

def scrape_article(url):
    """Your original working scraper"""
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        article_tag = soup.find("article")
        time_tag = article_tag.find("time", datetime=True) if article_tag else None
        author_tags = soup.select("a[href^='/authors/']")


        content = []
        if article_tag:
            for tag in article_tag.find_all(["h1", "h2", "h3", "p", "blockquote", "ul", "ol"]):
                text = tag.get_text(" ", strip=True)
                if text:
                    content.append(text)

        return {
            "url": url,
            "date_time": time_tag["datetime"] if time_tag else None,
            "author": ", ".join([a.get_text(strip=True) for a in author_tags]) or "Unknown",
            "content": "\n\n".join(content) if content else None
        }

    except Exception as e:
        return {"url": url, "error": str(e)}

def main():
    """Parallel version with Parquet handling and ETA"""
    start_time = time.time()


    df = pd.read_parquet(INPUT_FILE)
    urls = df["url"].tolist()
    total = len(urls)

    results = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(scrape_article, url): url for url in urls}

        processed = 0
        for future in as_completed(futures):
            processed += 1
            result = future.result()
            if result.get('content'):
                results.append(result)

            elapsed = time.time() - start_time
            avg_time = elapsed / processed
            remaining = total - processed
            eta = format_eta(avg_time * remaining)

            sys.stdout.write(
                f"\r🚀 Processed: {processed}/{total} | "
                f"✅ Valid: {len(results)} | "
                f"⏳ ETA: {eta} | "
                f"Elapsed: {format_eta(elapsed)}"
            )
            sys.stdout.flush()


    pd.DataFrame(results).to_parquet(OUTPUT_FILE, index=False)
    print(f"\n\n✅ Saved {len(results)} articles to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

🚀 Processed: 4515/4515 | ✅ Valid: 4515 | ⏳ ETA: 00:00:00 | Elapsed: 00:05:31

✅ Saved 4515 articles to /content/scraped_vox_articles.parquet


In [ ]:
import pandas as pd

file_path = "/content/scraped_vox_articles.parquet"
df = pd.read_parquet(file_path)


print(df.head())


                                                 url  \
0  https://www.vox.com/videos/2023/10/31/23940573...   
1  https://www.vox.com/23930119/hate-crimes-musli...   
2  https://www.vox.com/world-politics/2023/10/31/...   
3  https://www.vox.com/technology/2023/10/31/2393...   
4  https://www.vox.com/world-politics/2023/10/31/...   

                   date_time                          author  \
0  2023-10-31T18:50:32+00:00        Edward Vega, Edward Vega   
1  2023-10-31T15:40:06+00:00  Fabiola Cineas, Fabiola Cineas   
2  2023-10-31T19:00:00+00:00                Li Zhou, Li Zhou   
3  2023-10-31T18:45:43+00:00    Sara Morrison, Sara Morrison   
4  2023-10-31T15:20:00+00:00  Zack Beauchamp, Zack Beauchamp   

                                             content  
0  Video\n\nHow The Conjuring became the Marvel o...  
1  Politics / World Politics\n\n“History repeatin...  
2  Politics / World Politics\n\nWhat we know abou...  
3  Technology\n\nPresident Biden’s new plan to re...  
4  

In [ ]:
import pandas as pd
import re

def clean_data_preview():
    try:
        df = pd.read_parquet("/content/scraped_vox_articles.parquet")

        df['date_time'] = pd.to_datetime(df['date_time'], errors='coerce')
        df['date'] = df['date_time'].dt.strftime('%Y-%m-%d')
        df['time'] = df['date_time'].dt.strftime('%H:%M UTC')


        def parse_author_category(text):
            if pd.isna(text):
                return ('Unknown', 'General')


            parts = re.split(r'\n\n|: ', text, 1)
            if len(parts) > 1:
                author = parts[0].split(',')[0].strip()
                category = parts[1].split('\n')[0].strip()
                return (author, category)
            return (text.split(',')[0].strip(), 'General')

        df[['author', 'category']] = df['author'].apply(
            lambda x: pd.Series(parse_author_category(x))
        )


        df['clean_content'] = df['content'].str.replace('\n', ' ', regex=True).str.strip()

        preview = df.head().copy()
        preview['content_preview'] = preview['clean_content'].str.slice(0, 150) + '...'


        print(f"📆 Articles from {df['date'].min()} to {df['date'].max()}")
        print("\n🧑💻 Authors & Categories:")
        print(preview[['date', 'time', 'author', 'category']].to_string(index=False))

        print("\n📰 Content Previews:")
        for i, row in preview.iterrows():
            print(f"\n=== {row['date']} {row['time']} [{row['category']}] ===")
            print(f"AUTHOR: {row['author']}")
            print(f"CONTENT: {row['content_preview']}\n")

    except Exception as e:
        print(f"🚨 Error: {str(e)}")

clean_data_preview()

📆 Articles from 2023-01-01 to 2025-02-17

🧑💻 Authors & Categories:
      date      time         author category
2023-10-31 18:50 UTC    Edward Vega  General
2023-10-31 15:40 UTC Fabiola Cineas  General
2023-10-31 19:00 UTC        Li Zhou  General
2023-10-31 18:45 UTC  Sara Morrison  General
2023-10-31 15:20 UTC Zack Beauchamp  General

📰 Content Previews:

=== 2023-10-31 18:50 UTC [General] ===
AUTHOR: Edward Vega
CONTENT: Video  How The Conjuring became the Marvel of horror  The Warren’s case files have helped create a shockingly successful horror universe.  by Edward V...


=== 2023-10-31 15:40 UTC [General] ===
AUTHOR: Fabiola Cineas
CONTENT: Politics / World Politics  “History repeating itself”: How the Israel-Hamas war is fueling hate against Muslims and Jews  There’s a surge in reports o...


=== 2023-10-31 19:00 UTC [General] ===
AUTHOR: Li Zhou
CONTENT: Politics / World Politics  What we know about the ongoing hostage crisis in Gaza  More than 200 people were taken hostage by H

In [ ]:
import os
import glob
import pandas as pd

# load or create stock_df
stock_file_path = "merged_stock_data.parquet"
if os.path.exists(stock_file_path):
    stock_df = pd.read_parquet(stock_file_path)
    print(f"loaded stock data from {stock_file_path}")
else:
    csv_files = glob.glob(os.path.join("yfinance_tech_stock_data", "*.csv"))
    if not csv_files:
        raise FileNotFoundError("no merged stock parquet file or csv files found")
    stock_df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    print(f"merged {len(csv_files)} csv files")
    stock_df.to_parquet(stock_file_path, index=False)
    print(f"saved merged stock data to {stock_file_path}")

# load news_df
news_file_path = "scraped_vox_articles.parquet"
if os.path.exists(news_file_path):
    news_df = pd.read_parquet(news_file_path)
    print(f"loaded news data from {news_file_path}")
else:
    raise FileNotFoundError(f"news data file not found")

# rename date_time to timestamp
if 'timestamp' not in news_df.columns:
    if 'date_time' in news_df.columns:
        news_df.rename(columns={'date_time': 'timestamp'}, inplace=True)
        print("renamed date_time to timestamp")
    else:
        raise KeyError("neither timestamp nor date_time found")

# create headline and article_text if not present
if 'headline' not in news_df.columns:
    if 'content' in news_df.columns:
        news_df['headline'] = news_df['content'].apply(lambda x: x.split('.')[0] if isinstance(x, str) and '.' in x else x)
        print("created headline from content")
    else:
        news_df['headline'] = None

if 'article_text' not in news_df.columns:
    if 'content' in news_df.columns:
        news_df['article_text'] = news_df['content']
        print("created article_text from content")
    else:
        news_df['article_text'] = None

# ensure correct column names in stock_df
if 'price' not in stock_df.columns and 'Close' in stock_df.columns:
    stock_df.rename(columns={'Close': 'price'}, inplace=True)
if 'volume' not in stock_df.columns and 'Volume' in stock_df.columns:
    stock_df.rename(columns={'Volume': 'volume'}, inplace=True)
if 'timestamp' not in stock_df.columns:
    if 'Date' in stock_df.columns:
        stock_df.rename(columns={'Date': 'timestamp'}, inplace=True)
    else:
        raise KeyError("no timestamp or Date column found")

# convert timestamps to datetime utc
stock_df['timestamp'] = pd.to_datetime(stock_df['timestamp'], errors='coerce', utc=True)
news_df['timestamp'] = pd.to_datetime(news_df['timestamp'], errors='coerce', utc=True)

# proceed with merging
try:
    # check required columns
    required_stock_columns = ['timestamp', 'price', 'volume']
    required_news_columns = ['timestamp', 'headline', 'article_text']

    for col in required_stock_columns:
        if col not in stock_df.columns:
            raise KeyError(f"column {col} missing from stock_df")
    for col in required_news_columns:
        if col not in news_df.columns:
            raise KeyError(f"column {col} missing from news_df")

    # create time buckets
    bucket_size = 'H'
    stock_df['time_bucket'] = stock_df['timestamp'].dt.floor(bucket_size)
    news_df['time_bucket'] = news_df['timestamp'].dt.floor(bucket_size)

    # aggregate news articles
    news_bucketed = news_df.groupby('time_bucket').agg({
        'headline': lambda x: ' | '.join(x.dropna()),
        'article_text': lambda x: ' '.join(x.dropna())
    }).reset_index()

    # merge stock with news
    merged_df = pd.merge(stock_df, news_bucketed, on='time_bucket', how='left', suffixes=('_stock', '_news'))

    if merged_df.empty:
        print("warning: merged dataframe is empty")
    else:
        print("merged dataframe (first 5 rows):")
        print(merged_df.head())
        print("merged dataframe info:")
        print(merged_df.info())

except Exception as e:
    print("error during merging:")
    print(e)

loaded stock data from merged_stock_data.parquet
loaded news data from scraped_vox_articles.parquet
renamed date_time to timestamp
created headline from content
created article_text from content


<ipython-input-13-02a08782f495>:80: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  stock_df['time_bucket'] = stock_df['timestamp'].dt.floor(bucket_size)
<ipython-input-13-02a08782f495>:81: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  news_df['time_bucket'] = news_df['timestamp'].dt.floor(bucket_size)


merged dataframe (first 5 rows):
                  timestamp       Open       High        Low      price  \
0 2023-02-15 14:30:00+00:00  40.174999  40.470001  40.174999  40.424999   
1 2023-02-15 15:30:00+00:00  40.424999  40.590000  40.299999  40.584999   
2 2023-02-15 16:30:00+00:00  40.580002  40.619999  40.424999  40.424999   
3 2023-02-15 17:30:00+00:00  40.419998  40.529999  40.390049  40.490002   
4 2023-02-15 18:30:00+00:00  40.494999  40.680000  40.470001  40.619999   

   volume  Dividends  Stock Splits symbol               time_bucket  \
0  492952        0.0           0.0    APH 2023-02-15 14:00:00+00:00   
1  448717        0.0           0.0    APH 2023-02-15 15:00:00+00:00   
2  109341        0.0           0.0    APH 2023-02-15 16:00:00+00:00   
3  134195        0.0           0.0    APH 2023-02-15 17:00:00+00:00   
4  163544        0.0           0.0    APH 2023-02-15 18:00:00+00:00   

                                            headline  \
0    Life / Even Better\n\nYou’re